# HW10

 ## Question 13.2  

In this problem you, can simulate a simplified airport security system at a busy airport. Passengers arrive according to a Poisson distribution with $\lambda_1$ = 5 per minute (i.e., mean interarrival rate $\mu_1$ = 0.2 minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate $\mu_2$ = 0.75 minutes. [Hint: model them as one block that has more than one resource.]  After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute). 

Use the Arena software (PC users) or Python with SimPy (PC or Mac users) to build a simulation of the system, and then vary the number of ID/boarding-pass checkers and personal-check queues to determine how many are needed to keep average wait times below 15 minutes.  [If you’re using SimPy, or if you have access to a non-student version of Arena, you can use λ1 = 50 to simulate a busier airport.]

----- OUR MODEL -----  
Passenger arrives --> Queue 1: ID/boarding check --> Queue 2: (shortest) Personal check --> Plane  
  

In [1]:
# see https://pythonhosted.org/SimPy/Tutorials/TheBank.html
import simpy
import random

## Experiment Data ------------------------------

# Given information
numIterations = 50 # number of simulation iterations
arrivalRate = 5 # given in instructions
checkRate = 0.75
scanRate_low = 0.5
scanRate_high = 1.0
endSimTime = 500

# We're trying to optimize the number of Checkers and Scanners
Checkers = 5
Scanners = 4


ModuleNotFoundError: No module named 'simpy'

In [ ]:
## Model Components ------------------------------  

# create the total Airport system with checkers and scanners. Define time spent at each.
class Airport(object):
    def __init__(self, environment): #, numCheckers, numScanners):
        self.environment = environment
        
        # set up number of checkers
        self.checker = simpy.Resource(environment, capacity = Checkers)

        # set up scanners; each has its own queue
        self.scanner = []
        
        # for each scanner, assign assign a resource
        for i in range(Scanners):
            res_scan = simpy.Resource(environment, capacity = 1)
            self.scanner.append(res_scan)

    # time to check a passenger
    def checkTime(self, passenger):
        yield self.environment.timeout(random.expovariate(1.0/checkRate))
        
    # time to scan a passenger
    def scanTime(self, passenger):
        yield self.environment.timeout(random.uniform(scanRate_low, scanRate_high))
        

# each passenger has an input of: 
# current environment, Airport scenario, passenger number
def passenger(environment, Airport, pass_num):
    global pass_count # number of passengers going through the Airport
    global totalTime # total time spent in the Airport
    # record the time the passenger arrives
    timeAtArrival = environment.now

    # see https://simpy.readthedocs.io/en/latest/topical_guides/resources.html
    with Airport.checker.request() as req:
        # wait for access
        yield req 
        timeCheckIn = environment.now
        
        # passenger waits in line
        yield environment.process(Airport.checkTime(pass_num))
        # resource is released automatically(?)
        
        # note the time the passenger gets to the checker
        timeCheckOut = environment.now
        # wait time upon Arrival is the difference
        checkTime_i = (timeCheckOut - timeCheckIn)
        # append the checkTime_i to the global checkTime array
        checkTime.append(checkTime_i)
    
    # After the checker, the passenger looks for the shortest queue among the scanners.
    # Iterate through the scanners and check their queue. Find the shortest one.
    shorter_queue = 0
    for s in range(1, Scanners): # numScanners):
        # if the length of the scanner is less than the length of 'shorter_queue', 
        if (len(Airport.scanner[s].queue) < len(Airport.scanner[shorter_queue].queue)):
            shorter_queue = s
        
    # passenger goes to the scanner queue
    with Airport.scanner[shorter_queue].request() as req:
        # wait for access
        yield req
        
        # record the time to reach the scanner
        timeScanIn = environment.now
        
        # passenger waits in line
        yield environment.process(Airport.scanTime(pass_num))
        
        # record the time to leave the scanner; calculate time at the scanner
        timeScanOut = environment.now
        scanTime_i = timeScanOut - timeScanIn
        # append the scanTime to the global scanTime array
        scanTime.append(scanTime_i)
    
    # record the time the passenger leaves the airport; calculate total time at the airport
    timeAtDepart = environment.now
    totalTime_i = timeAtDepart - timeAtArrival
    # append totalTime_i to the global totalTime array
    totalTime.append(totalTime_i)
    
    # add 1 to the total count of passengers that have gone through the airport
    pass_count += 1        

# lay out the arrival for each passenger, and record the time(s) they are in the airport
def arrival(environment): 
    a = Airport(environment)
    i = 0
    while True:     
        yield environment.timeout(random.expovariate(arrivalRate))
        # add a passenger in the count
        i+=1
        
        # passenger goes through the airport
        # the a'th scenario; the i'th passenger
        environment.process(passenger(environment, a, i)) 

In [ ]:
## Experiment Data ------------------------------

# Set up empty dataframes to keep track of all times for different combinations
import pandas as pd
import numpy as np

avgTimes = pd.DataFrame(0, index=np.arange(numIterations), columns=[i for i in range(4)])
avgTimes.columns = ['Avg Check Time','Avg Scan Time', 'Avg Total Time', 'Avg Wait Time']

In [ ]:
## Experiment / Result ------------------------------

# see https://simpy.readthedocs.io/en/latest/topical_guides/resources.html
# Iterate through the possible simulations. 

for i in range(numIterations):
    
    random.seed(i)

    # reset variables for each simulation: total Passengers, check time, scanner time, total time
    pass_count = 0
    checkTime =[] 
    scanTime =[] 
    totalTime  = []
    
    # start simulation - https://simpy.readthedocs.io/en/latest/topical_guides/resources.html
    env = simpy.Environment()
    env.process(arrival(env))
    env.run(until=endSimTime) # must put something here, or else it never stops
      
    # Track durations of simulation i, input those values into the avgTimes dataframe.
    avgTimes.iloc[i , 0] = sum(checkTime[1:pass_count]) / pass_count  # checker time
    avgTimes.iloc[i , 1] = sum(scanTime[1:pass_count]) / pass_count  # scanner time
    avgTimes.iloc[i , 2] = sum(totalTime[1:pass_count]) / pass_count  # total time
    
    # Wait time = (Total time in the airport) - (time at the Checker and time at the Scanner)
    avgTimes.iloc[i , 3] = avgTimes.iloc[i, 2] - (avgTimes.iloc[i, 0] + avgTimes.iloc[i, 1])

# Check the dataframe for this combination of checkers and Scanners
avgTimes.head()

In [ ]:
# calculate average wait times
avgTimes['Avg Wait Time'].mean()

We iterate this procedure, starting with 2 Checkers and 2 Scanners. We add one increment to each until we each less than 15 minutes for Wait Time. Below are our results.  
  
Checkers, Scanners, and Wait Times:  
2, 2, 179.92  
2, 3, 174.08  
3, 2, 174.29  
3, 3, 82.59  
3, 4, 76.46  
4, 3, 75.33  
4, 4, 4.01  
4, 5, 2.96  
5, 4, 1.79
  
We find that having 4 Checkers and 4 Scanners results in an average wait time of over 4 minutes.